# Preprare configs

This notebook supports the process of preparing config files.
The goals of the notebooks are as follows:
- extract selected frame(s) from the video
- define and test unwrapping parameters (camera view -> bird's eye view)
- define brid-eye to map transition parameters
- check the lane mask

In [ ]:
import sys
sys.path.insert(0,'..')

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pathlib 

import pickle
from moveoverlib.functions import *

import config

%matplotlib inline

In [ ]:
pathlib.Path(config.DATA_PATH).mkdir(parents=True, exist_ok=True)

VIDEO_FILE = '../../videos/vid_source.mp4'

pickle.dump(VIDEO_FILE, open(f'{config.DATA_PATH}/videopath.p', 'wb'))

# Create folder for images
pathlib.Path('./images/').mkdir(parents=True, exist_ok=True)

# Extract frames
extractFrame(VIDEO_FILE, frameno = (0*60)*15, dest_file = './images/frame_view1.jpg');


Points for source (camera view) and destination (image view)
First four points are used for unwrapping, the rest is just display at tests

In [ ]:
SRC = np.float32([
    [581, 727], # Left speed limit
    [1458, 717], # Right speed limit
    [800, 430], # Left railing
    [1578, 411], # Right railing
    [643, 474], # Sign
])

DST = np.float32([
    [206, 29], # Left speed limit
    [41, 75], # Right speed limit
    [567, 925], # Left railing
    [287, 1170], # Right railing
    [543, 724], # Sign
])


print ('  "cameraPoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],'.format(
    int(SRC[0][0]), int(SRC[0][1]),
    int(SRC[1][0]), int(SRC[1][1]),
    int(SRC[2][0]), int(SRC[2][1]),
    int(SRC[3][0]), int(SRC[3][1])
))

print ('  "birdEyePoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],'.format(
    int(DST[0][0]), int(DST[0][1]),
    int(DST[1][0]), int(DST[1][1]),
    int(DST[2][0]), int(DST[2][1]),
    int(DST[3][0]), int(DST[3][1]),
))



In [ ]:
M = cv2.getPerspectiveTransform(SRC[:4], DST[:4])
Minv = cv2.getPerspectiveTransform(DST[:4], SRC[:4])

In [ ]:
map_file = './images/SkyView.jpg'
ret = displayPoints(SRC, M, './images/frame_view1.jpg', map_file)

In [ ]:
ret = displayPoints(DST, Minv, map_file,  './images/frame_view1.jpg')

# SkyView 2 Geocoordinates

In [ ]:
latlon1 = 45.743893, -122.660828
xy1 = 96, 30
latlon2 = 45.742497, -122.659864
xy2 = 635, 1151

# RITIS Data
Useful only in MoveOver2. For MoveOver1 project, just ignore it.

In [1]:
TMC_SEGMENT = ''
RITIS_SPEED = None
RITIS_BOTTLENECK = False

# Lane masks

In [ ]:
MASK_PATH = './images/mask.png'

sys.path.insert(0,'../..')
from cav.parameters import Parameters
params = Parameters()
params.generateParameters('./params.json')


mask = (255*plt.imread(MASK_PATH)).astype(int)
if (len(mask.shape) == 3) and (mask.shape[2] > 1):
    #mask = np.sum(mask, axis=2)
    mask = mask[:, :, 0]
plt.imshow(mask, cmap='gray')
unique = np.unique(mask, return_counts=True)
print (unique)

if [0] + params.lanes_mask == sorted(unique[0]):
    print ('OK! Mask parameers defined correctly in params.json.')
else:
    print (f'json file : {[0] + params.lanes_mask}')
    print (f'from image : {sorted(unique)}')
    

### Create json file

In [ ]:
img = cv2.imread('./images/frame_view1.jpg')
skyview = cv2.imread('./images/SkyView.jpg')

jsonfile = '''{{
  "videoShape" : [{}, {}],
  "birdEyeViewShape" : [{}, {}],

  "cameraPoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],
  "birdEyePoints" : [[{}, {}], [{}, {}], [{}, {}], [{}, {}]],

  "birdEyeCoordinates" : [[{}, {}], [{}, {}]],
  "latLonCoordinates" : [[{}, {}], [{}, {}]],
  "elevation" : 40,
    
  "lanes_mask" : {}
  "tmc_segment" : {}
  "ritis_speed" : {}
  "ritis_bottleneck" : {}
}}'''.format(
    img.shape[1], img.shape[0], # videoShape
    skyview.shape[1], skyview.shape[0], # birdEyeViewShape
    
    int(SRC[0][0]), int(SRC[0][1]), # cameraPoints
    int(SRC[1][0]), int(SRC[1][1]),
    int(SRC[2][0]), int(SRC[2][1]),
    int(SRC[3][0]), int(SRC[3][1]), # cameraPointsEnd 
    
    int(DST[0][0]), int(DST[0][1]), # birdEyePoints
    int(DST[1][0]), int(DST[1][1]),
    int(DST[2][0]), int(DST[2][1]),
    int(DST[3][0]), int(DST[3][1]), # birdEyePointsEnd
    
    xy1[0], xy1[1], xy2[0], xy2[1], #birdEyeCoordinates

    latlon1[1], latlon1[0], #latLonCoordinates
    latlon2[1], latlon2[0],
    
    str(sorted(unique[0])[1:]), # lanes_mask

    str(TMC_SEGMENT), # RITIS-based data
    RITIS_SPEED,
    RITIS_BOTTLENECK,
)

with open('params.json', 'w') as f:
    for line in jsonfile.split('\n'):
        print (line)
        f.write(line + '\n')

